In [1]:
# rm(list=ls())
require(data.table)
require(MASS)
require(ggplot2)
require(gridExtra)
require(stringr)
require(doParallel)
require(abind)
require(Matrix)
set.seed(123)

## Resizing notebook plot space
options(repr.plot.width=16, repr.plot.height=9)

Loading required package: data.table
Loading required package: MASS
Loading required package: ggplot2
Loading required package: gridExtra
Loading required package: stringr
Loading required package: doParallel
Loading required package: foreach
Loading required package: iterators
Loading required package: parallel
Loading required package: abind
Loading required package: Matrix


## (0) Some functions

In [2]:
## Load in the R functions
source("/home/j/temp/central_comp/libraries/current/r/get_draws.R")

In [3]:
## Neal's multivariate copula function

draw2Dcopula <- function(X, cor_mat, df_return = F){
    
    ## If X is a df, convert to array; else continue
    
  L <- dim(X)[2]
  D <- dim(X)[3]
  Xsum <- apply(X, c(2, 3), sum)
  mvdat <- mvrnorm(n=D, mu=0 * 1:L, Sigma=cor_mat, empirical=TRUE)
  ranks <- apply(mvdat, 2, rank, ties.method="first")
  sortedXsim <- apply(Xsum, 1, function(x) sort(x, index.return=TRUE)$ix)
  sortedX <- X
  for(i in 1:L){
    sortedX[,i,] <- X[,i,sortedXsim[,i]]
  }
  Xcorr <- sortedX
  for(i in 1:L){
    Xcorr[,i,] <- sortedX[,i,ranks[,i]]
  }
  if (df_return==T) {
    return(data.table(melt(Xcorr)))
    }
      else {
          Xcorr
      }
}


In [4]:
## Stacking function: stack the names of the ones that matter and ones that don't
stacking_df <- function(data, to_stack, to_stack_name, no_stack, no_stack_name) {
    
    data[,paste0(to_stack_name) := do.call(paste, c(data[, .SD, .SDcols = to_stack], sep = "_")) ]
    data[, paste0(no_stack_name) := do.call(paste, c(data[, .SD, .SDcols = no_stack], sep = "_"))]
    
    return(data)
}

## (1) Prepping the data to be into the array we want

In [5]:
## Get some example draws (causes 587 and 495)
dalys_diabetes <- get_draws(gbd_id_field = "cause_id", gbd_id = 587, source = 'dalynator', metric_id = 1,
                            measure_ids = 2, location_id = c(6,7,8), 
                            age_group_ids = c(6,7,8,9,10), sex_ids = c(1,2))

dalys_ihd <- get_draws(gbd_id_field = "cause_id", gbd_id = 495, source = 'dalynator', metric_id = 1,
                            measure_ids = 2, location_id = c(6,7,8), 
                            age_group_ids = c(6,7,8,9,10), sex_ids = c(1,2))

In [44]:
head(dalys_ihd)
names(dalys_ihd)[grepl("draw", colnames(dalys_ihd))]

location_id,year_id,age_group_id,sex_id,cause_id,draw_0,draw_1,draw_2,draw_3,draw_4,⋯,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,age_sex_cause,loc_year
6,2016,006,1,495,1237.081,1063.046,1092.417,1050.003,1164.845,⋯,1732.339,1684.839,1885.455,1675.191,1374.366,1335.179,1997.662,1757.772,006_1_495,6_2016
6,2016,007,1,495,1732.085,1460.538,1635.530,1767.736,1548.000,⋯,2227.595,2223.089,2949.874,2516.195,2206.735,1928.136,2527.165,2448.294,007_1_495,6_2016
6,2016,008,1,495,5674.693,4296.232,5089.813,5230.881,5192.324,⋯,5630.960,5572.219,6712.564,7226.864,6033.024,5805.579,6649.542,6964.594,008_1_495,6_2016
6,2016,009,1,495,13522.363,11248.563,11836.700,12138.455,12568.434,⋯,13790.955,13766.029,12968.592,19288.845,12602.224,11544.237,14062.304,12435.445,009_1_495,6_2016
6,2016,010,1,495,27353.899,24259.881,24763.184,26199.080,28350.986,⋯,30899.984,27327.438,29298.628,31618.218,29835.337,27912.920,29859.533,26978.516,010_1_495,6_2016
6,2016,006,2,495,1210.861,1191.965,1233.869,1162.365,1141.147,⋯,1797.301,1561.403,1725.167,1782.010,1524.672,1616.626,1680.108,1644.068,006_2_495,6_2016


[1] "draw_0"   "draw_1"   "draw_2"   "draw_3"   "draw_4"   "draw_5"  
   [7] "draw_6"   "draw_7"   "draw_8"   "draw_9"   "draw_10"  "draw_11" 
  [13] "draw_12"  "draw_13"  "draw_14"  "draw_15"  "draw_16"  "draw_17" 
  [19] "draw_18"  "draw_19"  "draw_20"  "draw_21"  "draw_22"  "draw_23" 
  [25] "draw_24"  "draw_25"  "draw_26"  "draw_27"  "draw_28"  "draw_29" 
  [31] "draw_30"  "draw_31"  "draw_32"  "draw_33"  "draw_34"  "draw_35" 
  [37] "draw_36"  "draw_37"  "draw_38"  "draw_39"  "draw_40"  "draw_41" 
  [43] "draw_42"  "draw_43"  "draw_44"  "draw_45"  "draw_46"  "draw_47" 
  [49] "draw_48"  "draw_49"  "draw_50"  "draw_51"  "draw_52"  "draw_53" 
  [55] "draw_54"  "draw_55"  "draw_56"  "draw_57"  "draw_58"  "draw_59" 
  [61] "draw_60"  "draw_61"  "draw_62"  "draw_63"  "draw_64"  "draw_65" 
  [67] "draw_66"  "draw_67"  "draw_68"  "draw_69"  "draw_70"  "draw_71" 
  [73] "draw_72"  "draw_73"  "draw_74"  "draw_75"  "draw_76"  "draw_77" 
  [79] "draw_78"  "draw_79"  "draw_80"  "draw_81"  "draw_82"  "draw_83" 
  [85] "draw_84"  "draw_85"  "draw_86"  "draw_87"  "draw_88"  "draw_89" 
  [91] "draw_90"  "draw_91"  "draw_92"  "draw_93"  "draw_94"  "draw_95" 
  [97] "draw_96"  "draw_97"  "draw_98"  "draw_99"  "draw_100" "draw_101"
 [103] "draw_102" "draw_103" "draw_104" "draw_105" "draw_106" "draw_107"
 [109] "draw_108" "draw_109" "draw_110" "draw_111" "draw_112" "draw_113"
 [115] "draw_114" "draw_115" "draw_116" "draw_117" "draw_118" "draw_119"
 [121] "draw_120" "draw_121" "draw_122" "draw_123" "draw_124" "draw_125"
 [127] "draw_126" "draw_127" "draw_128" "draw_129" "draw_130" "draw_131"
 [133] "draw_132" "draw_133" "draw_134" "draw_135" "draw_136" "draw_137"
 [139] "draw_138" "draw_139" "draw_140" "draw_141" "draw_142" "draw_143"
 [145] "draw_144" "draw_145" "draw_146" "draw_147" "draw_148" "draw_149"
 [151] "draw_150" "draw_151" "draw_152" "draw_153" "draw_154" "draw_155"
 [157] "draw_156" "draw_157" "draw_158" "draw_159" "draw_160" "draw_161"
 [163] "draw_162" "draw_163" "draw_164" "draw_165" "draw_166" "draw_167"
 [169] "draw_168" "draw_169" "draw_170" "draw_171" "draw_172" "draw_173"
 [175] "draw_174" "draw_175" "draw_176" "draw_177" "draw_178" "draw_179"
 [181] "draw_180" "draw_181" "draw_182" "draw_183" "draw_184" "draw_185"
 [187] "draw_186" "draw_187" "draw_188" "draw_189" "draw_190" "draw_191"
 [193] "draw_192" "draw_193" "draw_194" "draw_195" "draw_196" "draw_197"
 [199] "draw_198" "draw_199" "draw_200" "draw_201" "draw_202" "draw_203"
 [205] "draw_204" "draw_205" "draw_206" "draw_207" "draw_208" "draw_209"
 [211] "draw_210" "draw_211" "draw_212" "draw_213" "draw_214" "draw_215"
 [217] "draw_216" "draw_217" "draw_218" "draw_219" "draw_220" "draw_221"
 [223] "draw_222" "draw_223" "draw_224" "draw_225" "draw_226" "draw_227"
 [229] "draw_228" "draw_229" "draw_230" "draw_231" "draw_232" "draw_233"
 [235] "draw_234" "draw_235" "draw_236" "draw_237" "draw_238" "draw_239"
 [241] "draw_240" "draw_241" "draw_242" "draw_243" "draw_244" "draw_245"
 [247] "draw_246" "draw_247" "draw_248" "draw_249" "draw_250" "draw_251"
 [253] "draw_252" "draw_253" "draw_254" "draw_255" "draw_256" "draw_257"
 [259] "draw_258" "draw_259" "draw_260" "draw_261" "draw_262" "draw_263"
 [265] "draw_264" "draw_265" "draw_266" "draw_267" "draw_268" "draw_269"
 [271] "draw_270" "draw_271" "draw_272" "draw_273" "draw_274" "draw_275"
 [277] "draw_276" "draw_277" "draw_278" "draw_279" "draw_280" "draw_281"
 [283] "draw_282" "draw_283" "draw_284" "draw_285" "draw_286" "draw_287"
 [289] "draw_288" "draw_289" "draw_290" "draw_291" "draw_292" "draw_293"
 [295] "draw_294" "draw_295" "draw_296" "draw_297" "draw_298" "draw_299"
 [301] "draw_300" "draw_301" "draw_302" "draw_303" "draw_304" "draw_305"
 [307] "draw_306" "draw_307" "draw_308" "draw_309" "draw_310" "draw_311"
 [313] "draw_312" "draw_313" "draw_314" "draw_315" "draw_316" "draw_317"
 [319] "draw_318" "draw_319" "draw_320" "draw_321" "draw_322" "draw_323"
 [325] "draw_324" "draw_325" "draw_326" "draw_327" "draw_328"

In [6]:
## Clean up and only what we need (loc, year, age, sex, draws):
dalys_ihd <- dalys_ihd[, .SD, .SDcols = c("location_id", "year_id", "age_group_id", "sex_id", "cause_id", paste0("draw_",c(0:999)) )]
dalys_diabetes <- dalys_diabetes[, .SD, .SDcols = c("location_id", "year_id", "age_group_id", "sex_id", "cause_id", paste0("draw_",c(0:999)) )]

In [8]:
## Convert age group to a string with leading zeroes, so that the sorting is maintained in the arrays
dalys_ihd[, age_group_id:= formatC(age_group_id, width = 3, format = "d", flag = "0")]
dalys_diabetes[, age_group_id:= formatC(age_group_id, width = 3, format = "d", flag = "0")]

In [52]:
paste(corring_over, collapse = 
      "__")

[1] "age_group_id__sex_id__cause_id"

In [9]:
## Stack age_sex and loc_year
corring_over <- c("age_group_id", "sex_id", "cause_id")
corring_var_name <- "age_sex_cause"


### NOTE: this can be automated: look at the setdiff of corring_ver and draw names
not_corring_over <- c("location_id", "year_id")
not_corring_var_name <- "loc_year"

dalys_ihd <- stacking_df(dalys_ihd, to_stack =  corring_over, to_stack_name = corring_var_name,
                         no_stack = not_corring_over, no_stack_name = not_corring_var_name)

dalys_diabetes <- stacking_df(dalys_diabetes, to_stack =  corring_over, to_stack_name = corring_var_name,
                         no_stack = not_corring_over, no_stack_name = not_corring_var_name)

In [10]:
head(dalys_diabetes); tail(dalys_ihd)

location_id,year_id,age_group_id,sex_id,cause_id,draw_0,draw_1,draw_2,draw_3,draw_4,⋯,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,age_sex_cause,loc_year
6,2016,006,1,587,1502.603,1464.278,1159.717,1123.799,1117.349,⋯,1749.500,1556.551,1537.333,1938.616,1579.557,1576.566,1485.885,1702.443,006_1_587,6_2016
6,2016,007,1,587,4544.980,5109.784,4331.432,4388.135,4026.030,⋯,7566.693,5161.448,4875.295,6053.514,4516.764,5316.403,5184.320,5352.031,007_1_587,6_2016
6,2016,008,1,587,22877.532,21976.269,16302.054,16337.912,16569.428,⋯,26773.150,20345.621,16477.317,20851.499,14993.269,17000.141,20112.642,16799.632,008_1_587,6_2016
6,2016,009,1,587,99663.020,85973.618,53148.500,54046.493,63074.437,⋯,83049.148,79542.014,68065.248,83753.853,46989.948,60156.015,61445.950,52345.268,009_1_587,6_2016
6,2016,010,1,587,207400.937,187044.589,116781.610,112546.733,133283.330,⋯,181121.544,170419.012,157672.722,193634.411,91804.074,137670.104,120549.784,122673.684,010_1_587,6_2016
6,2016,006,2,587,2129.917,1870.235,1835.528,1770.460,1846.798,⋯,2871.924,2508.616,2421.117,2665.646,2356.602,2619.106,2224.157,2340.772,006_2_587,6_2016


location_id,year_id,age_group_id,sex_id,cause_id,draw_0,draw_1,draw_2,draw_3,draw_4,⋯,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,age_sex_cause,loc_year
8,2005,010,1,495,233.988585,171.487190,209.91860,154.464707,135.579504,⋯,167.509717,174.953283,217.349300,140.951074,101.940970,205.146271,118.190764,233.869982,010_1_495,8_2005
8,2005,006,2,495,4.457144,6.132533,6.77355,7.412024,5.990928,⋯,5.085523,3.212157,8.599625,5.393792,4.949043,5.543227,5.685739,6.499283,006_2_495,8_2005
8,2005,007,2,495,11.385647,12.958085,13.54328,10.741678,8.925954,⋯,14.483783,11.895921,15.118872,15.822339,14.795711,11.584933,11.505027,13.801689,007_2_495,8_2005
8,2005,008,2,495,29.771521,31.065605,23.15046,22.519256,36.584111,⋯,20.610921,16.632874,30.001893,23.911072,24.180766,32.952451,30.363117,34.850801,008_2_495,8_2005
8,2005,009,2,495,128.309223,82.572138,85.95811,72.925669,69.914348,⋯,83.147467,74.205571,91.836777,101.789327,59.807383,82.291100,105.698706,122.497104,009_2_495,8_2005
8,2005,010,2,495,159.220627,158.660794,126.50677,161.854878,156.890885,⋯,144.133512,176.340762,124.226775,152.610968,129.468943,146.645636,127.134103,169.171297,010_2_495,8_2005


In [11]:
## Melt the draws and add a column called "var_name"
dalys_ihd_long <- melt(dalys_ihd, id.vars = c(not_corring_var_name, corring_var_name), 
                       measure.vars = paste0("draw_", c(0:999)),
                      value.name = "dalys", variable.name = "draw_num")

dalys_diabetes_long <- melt(dalys_diabetes, id.vars =c(not_corring_var_name, corring_var_name), 
                            measure.vars = paste0("draw_", c(0:999)),
                            value.name = "dalys", variable.name = "draw_num")

In [12]:
## Bind the dataframes
dalys_binded_long <- rbind(dalys_diabetes_long, dalys_ihd_long)

In [13]:
head(dalys_binded_long); tail(dalys_binded_long)

loc_year,age_sex_cause,draw_num,dalys
6_2016,006_1_587,draw_0,1502.603
6_2016,007_1_587,draw_0,4544.980
6_2016,008_1_587,draw_0,22877.532
6_2016,009_1_587,draw_0,99663.020
6_2016,010_1_587,draw_0,207400.937
6_2016,006_2_587,draw_0,2129.917


loc_year,age_sex_cause,draw_num,dalys
8_2005,010_1_495,draw_999,233.869982
8_2005,006_2_495,draw_999,6.499283
8_2005,007_2_495,draw_999,13.801689
8_2005,008_2_495,draw_999,34.850801
8_2005,009_2_495,draw_999,122.497104
8_2005,010_2_495,draw_999,169.171297


## (2) Devising AR correlation matrix for each unique corring variable (age, sex and cause) and then combine... maybe?

In [14]:
## Number of groups
age_groups <- unique(dalys_ihd[year_id == 2016, age_group_id])
sex_groups <- unique(dalys_ihd[year_id == 2016, sex_id])
year_groups <- unique(dalys_ihd[year_id == 2016, year_id])
loc_groups <- unique(dalys_ihd[year_id == 2016, location_id])
var_groups <- unique(c("heart_stuff", "diabeetus"))

In [15]:
## Matrices within each group first (2x2 matrices do not follow AR processes)
age_corr_mat <- 0.75**abs(outer(1:length(age_groups), 1:length(age_groups), "-"))
colnames(age_corr_mat) = rownames(age_corr_mat) = age_groups

sex_corr_mat <- 0.4**abs(outer(1:length(sex_groups), 1:length(sex_groups), "-"))
colnames(sex_corr_mat) = rownames(sex_corr_mat) = sex_groups

var_corr_mat <- 0.6**abs(outer(1:length(var_groups), 1:length(var_groups), "-"))
colnames(var_corr_mat) = rownames(var_corr_mat) = var_groups

In [19]:
age_corr_mat

,006,007,008,009,010
006,1.0000000,0.750000,0.5625,0.421875,0.3164062
007,0.7500000,1.000000,0.7500,0.562500,0.4218750
008,0.5625000,0.750000,1.0000,0.750000,0.5625000
009,0.4218750,0.562500,0.7500,1.000000,0.7500000
010,0.3164062,0.421875,0.5625,0.750000,1.0000000


In [20]:
## Create the ultimate correlation matrix: the Kronecker product (IN THE REVERSE ORDER OF age_sex_var)
k1 <- kronecker(var_corr_mat, sex_corr_mat, make.dimnames = T)
k2 <- kronecker(k1, age_corr_mat, make.dimnames = T)

In [21]:
k2

,heart_stuff:1:006,heart_stuff:1:007,heart_stuff:1:008,heart_stuff:1:009,heart_stuff:1:010,heart_stuff:2:006,heart_stuff:2:007,heart_stuff:2:008,heart_stuff:2:009,heart_stuff:2:010,diabeetus:1:006,diabeetus:1:007,diabeetus:1:008,diabeetus:1:009,diabeetus:1:010,diabeetus:2:006,diabeetus:2:007,diabeetus:2:008,diabeetus:2:009,diabeetus:2:010
heart_stuff:1:006,1.0000000,0.750000,0.5625,0.421875,0.3164062,0.4000000,0.300000,0.2250,0.168750,0.1265625,0.6000000,0.450000,0.3375,0.253125,0.1898438,0.2400000,0.180000,0.1350,0.101250,0.0759375
heart_stuff:1:007,0.7500000,1.000000,0.7500,0.562500,0.4218750,0.3000000,0.400000,0.3000,0.225000,0.1687500,0.4500000,0.600000,0.4500,0.337500,0.2531250,0.1800000,0.240000,0.1800,0.135000,0.1012500
heart_stuff:1:008,0.5625000,0.750000,1.0000,0.750000,0.5625000,0.2250000,0.300000,0.4000,0.300000,0.2250000,0.3375000,0.450000,0.6000,0.450000,0.3375000,0.1350000,0.180000,0.2400,0.180000,0.1350000
heart_stuff:1:009,0.4218750,0.562500,0.7500,1.000000,0.7500000,0.1687500,0.225000,0.3000,0.400000,0.3000000,0.2531250,0.337500,0.4500,0.600000,0.4500000,0.1012500,0.135000,0.1800,0.240000,0.1800000
heart_stuff:1:010,0.3164062,0.421875,0.5625,0.750000,1.0000000,0.1265625,0.168750,0.2250,0.300000,0.4000000,0.1898438,0.253125,0.3375,0.450000,0.6000000,0.0759375,0.101250,0.1350,0.180000,0.2400000
heart_stuff:2:006,0.4000000,0.300000,0.2250,0.168750,0.1265625,1.0000000,0.750000,0.5625,0.421875,0.3164062,0.2400000,0.180000,0.1350,0.101250,0.0759375,0.6000000,0.450000,0.3375,0.253125,0.1898438
heart_stuff:2:007,0.3000000,0.400000,0.3000,0.225000,0.1687500,0.7500000,1.000000,0.7500,0.562500,0.4218750,0.1800000,0.240000,0.1800,0.135000,0.1012500,0.4500000,0.600000,0.4500,0.337500,0.2531250
heart_stuff:2:008,0.2250000,0.300000,0.4000,0.300000,0.2250000,0.5625000,0.750000,1.0000,0.750000,0.5625000,0.1350000,0.180000,0.2400,0.180000,0.1350000,0.3375000,0.450000,0.6000,0.450000,0.3375000
heart_stuff:2:009,0.1687500,0.225000,0.3000,0.400000,0.3000000,0.4218750,0.562500,0.7500,1.000000,0.7500000,0.1012500,0.135000,0.1800,0.240000,0.1800000,0.2531250,0.337500,0.4500,0.600000,0.4500000
heart_stuff:2:010,0.1265625,0.168750,0.2250,0.300000,0.4000000,0.3164062,0.421875,0.5625,0.750000,1.0000000,0.0759375,0.101250,0.1350,0.180000,0.2400000,0.1898438,0.253125,0.3375,0.450000,0.6000000


In [23]:
## The time has come: make the data.table into a multi dimensional array!
system.time(dalys_array <- reshape2::acast(dalys_binded_long, loc_year ~ age_sex_cause ~ draw_num, value.var = "dalys"))
str(dalys_array)

   user  system elapsed 
  0.959   0.033   0.993 

 num [1:18, 1:20, 1:1000] 6225 6932 5605 2492 2617 ...
 - attr(*, "dimnames")=List of 3
  ..$ : chr [1:18] "6_1990" "6_1995" "6_2000" "6_2005" ...
  ..$ : chr [1:20] "006_1_495" "006_1_587" "006_2_495" "006_2_587" ...
  ..$ : chr [1:1000] "draw_0" "draw_1" "draw_2" "draw_3" ...


In [24]:
## Copulate all over my body
system.time(dalys_corr <- draw2Dcopula(X = dalys_array, cor_mat = k2, df_return = T))
names(dalys_corr) <- colnames(dalys_binded_long)
str(dalys_corr)

   user  system elapsed 
  0.456   0.009   0.536 

Classes ‘data.table’ and 'data.frame':	360000 obs. of  4 variables:
 $ loc_year     : Factor w/ 18 levels "6_1990","6_1995",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ age_sex_cause: Factor w/ 20 levels "006_1_495","006_1_587",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ draw_num     : Factor w/ 1000 levels "draw_0","draw_1",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ dalys        : num  5313 6298 5245 2671 2533 ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [38]:
head(dalys_binded_long)

loc_year,age_sex_cause,draw_num,dalys
6_2016,006_1_587,draw_0,1502.603
6_2016,007_1_587,draw_0,4544.980
6_2016,008_1_587,draw_0,22877.532
6_2016,009_1_587,draw_0,99663.020
6_2016,010_1_587,draw_0,207400.937
6_2016,006_2_587,draw_0,2129.917
